<a href="https://colab.research.google.com/github/aamanirp/Replica/blob/main/Destination%20Cluster%20Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table



project = 'replica-customer' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

# for MapClusters

import os
import folium
from folium.plugins import MarkerCluster
import pandas as pd

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=replica-customer:US:bquxjob_4abc3503_189757f7505)
back to BigQuery to edit the query within the BigQuery user interface.

In [2]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_4abc3503_189757f7505') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT destination_bgrp_lat, destination_bgrp_lng
FROM `replica-customer.south_central.south_central_2021_Q4_thursday_trip` 
WHERE mode ="COMMERCIAL" AND distance_miles>15


# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [7]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_7d9088fe_189894af845') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

,destination_bgrp_lat,destination_bgrp_lng
0,32.370759,-90.769606
1,32.403515,-88.481746
2,31.991480,-89.253290
3,36.674815,-86.481996
4,32.536702,-92.009574
...,...,...
291469,35.023075,-89.905977
291470,35.023075,-89.905977
291471,35.023075,-89.905977
291472,35.023075,-89.905977


# New Section

### Map Marker Clusters


In [9]:

#Create a map
map_center = [results['destination_bgrp_lat'].iloc[0], results['destination_bgrp_lng'].iloc[0]]
map_cluster=folium.Map(location=map_center, zoom_start=5)


In [10]:

#Map cluster group
marker_cluster=MarkerCluster().add_to(map_cluster)


In [11]:
markers=[]
results.apply(lambda row: folium.Marker([row['destination_bgrp_lat'], row['destination_bgrp_lng']],
                                   popup=f"Lat: {row['destination_bgrp_lat']}<br>Lon: {row['destination_bgrp_lng']}")
               .add_to(marker_cluster), axis=1)

0         <folium.map.Marker object at 0x7af4c3efc100>
1         <folium.map.Marker object at 0x7af4c3eff4f0>
2         <folium.map.Marker object at 0x7af4c3efd8d0>
3         <folium.map.Marker object at 0x7af4c3efca30>
4         <folium.map.Marker object at 0x7af4c3f63f70>
                              ...                     
291469    <folium.map.Marker object at 0x7af4608cf5b0>
291470    <folium.map.Marker object at 0x7af4608cf6d0>
291471    <folium.map.Marker object at 0x7af4608cf7f0>
291472    <folium.map.Marker object at 0x7af4608cf910>
291473    <folium.map.Marker object at 0x7af4608cfa30>
Length: 291474, dtype: object

In [12]:
for marker in markers:
  marker.add_to(marker_cluster)

In [13]:
map_cluster.save("cluster_map.html")